# 스킨푸드 + javascript + subCategory

In [8]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urllib
from selenium import webdriver
from slacker import Slacker

In [9]:
slack = Slacker(token)

In [2]:
url = "http://www.theskinfood.com/"

In [3]:
category = pd.read_csv("skinfood_category.csv", encoding="utf-8")

In [4]:
category.head(3)

,title,div,cate,cate_link,cate_flag
0,SKINCARE,유형별,클렌징,goods/goodsList.do?catCd=4000000000&flag=40000...,4000000001
1,SKINCARE,유형별,팩/마스크,goods/goodsList.do?catCd=4000000000&flag=40000...,4000000009
2,SKINCARE,유형별,토너,goods/goodsList.do?catCd=4000000000&flag=40000...,4000000015


In [11]:
resultCate = []
num = 0
hub = 0
for i in range(101):
    ##Category값 저장
    title = category["title"][i]
    div = category["div"][i]
    cate = category["cate"][i]
    cate_link = category["cate_link"][i]
    cate_flag = category["cate_flag"][i]
    ##url생성
    sum_link = url + cate_link
    ##사이트 크롤링(자바스크립트 실행 전)
    driver = webdriver.Firefox()
    driver.get(sum_link)
        
    html = driver.page_source
    soup = BeautifulSoup(html)
    ##자바스크립트 찾아서 실행하기
    findMore = []
    findMore = soup.find_all("a", {"class":"more"})

    if "selectGoodsList" in findMore:
        driver.execute_script("selectGoodsList('','',10);")
    else:
        driver.execute_script(";")
    ##사이트 다시 크롤링(자바스크립트 실행 후)
    html = driver.page_source
    soup = BeautifulSoup(html)
    ##SubCategory 검사하기
    findCate = soup.find_all("ul", {"id" : cate_flag})
    ##SubCategory 정리하기
    findCate = str(findCate)
    findCate = findCate.replace("><", ">\n<")
    findCate = findCate.split("\n")
    ###findCate = html그대로의 subCategory
    driver.quit()
    ###subCate = a태그만 남겨놓은 subCategory
    subCate = []
    for line in findCate:
        if "a href" in line:
            subCate.append(str(line))
    ###totalSub = id값, 이름값으로 정리된 SubCategory
    totalSub = []
    for line in subCate:
        cate_id = line[36:46]
        cate_name = line[50:-4]
        id_name = [cate_id, cate_name]
        if id_name == []:
            continue
        totalSub.append(id_name)
    
    hub += len(totalSub)
    #totalCate = []
    ###cntSub = subCategory개수가 몇개인지 검사
    cntSub = len(totalSub)
    if cntSub == 0:
        final = [title, div, cate, cate_link, str(cate_flag), '0', '0' ]
        resultCate.append(final)
    for line in totalSub:
        sub_id = line[0]
        sub_cate = line[1]
        final = [title, div, cate, cate_link, str(cate_flag), sub_cate, str(sub_id)]
        resultCate.append(final)
    num+=1
    message = "Category sum SubCategory : " + str(num)
    slack.chat.post_message('#crawling', message)
    
    driver.quit()

/Users/SeulA/anaconda/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/SeulA/anaconda/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
/Users/SeulA/anaconda/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The co

In [12]:
resultCnt = len(resultCate) ##Result 146
resultCnt

146

In [13]:
len(resultCate) ##146
resultCate

[['SKINCARE',
  '유형별',
  '클렌징',
  'goods/goodsList.do?catCd=4000000000&flag=4000000001&gubun=B',
  '4000000001',
  '클렌징 폼',
  '4000000002'],
 ['SKINCARE',
  '유형별',
  '클렌징',
  'goods/goodsList.do?catCd=4000000000&flag=4000000001&gubun=B',
  '4000000001',
  '클렌징 오일',
  '4000000003'],
 ['SKINCARE',
  '유형별',
  '클렌징',
  'goods/goodsList.do?catCd=4000000000&flag=4000000001&gubun=B',
  '4000000001',
  '클렌징 로션/크림',
  '4000000004'],
 ['SKINCARE',
  '유형별',
  '클렌징',
  'goods/goodsList.do?catCd=4000000000&flag=4000000001&gubun=B',
  '4000000001',
  '클렌징 워터',
  '4000000005'],
 ['SKINCARE',
  '유형별',
  '클렌징',
  'goods/goodsList.do?catCd=4000000000&flag=4000000001&gubun=B',
  '4000000001',
  '클렌징 티슈',
  '4000000006'],
 ['SKINCARE',
  '유형별',
  '클렌징',
  'goods/goodsList.do?catCd=4000000000&flag=4000000001&gubun=B',
  '4000000001',
  '메이크업 리무버',
  '4000000007'],
 ['SKINCARE',
  '유형별',
  '클렌징',
  'goods/goodsList.do?catCd=4000000000&flag=4000000001&gubun=B',
  '4000000001',
  '비누',
  '4000000008'],
 ['SKI

In [17]:
result1 = resultCate

In [18]:
result2 = result1[0]
print(result2)

['SKINCARE', '유형별', '클렌징', 'goods/goodsList.do?catCd=4000000000&flag=4000000001&gubun=B', '4000000001', '클렌징 폼', '4000000002']


# Dataframe

In [19]:
df_columns = ["title", "div", "cate", "cate_link", "cate_flag", "sub_cate", "sub_id"]

In [20]:
cate_df = pd.DataFrame(columns=df_columns)

In [21]:
cate_df

,title,div,cate,cate_link,cate_flag,sub_cate,sub_id


In [22]:
reset = resultCate[1]
reset

['SKINCARE',
 '유형별',
 '클렌징',
 'goods/goodsList.do?catCd=4000000000&flag=4000000001&gubun=B',
 '4000000001',
 '클렌징 오일',
 '4000000003']

In [23]:
i=0
for i in range(resultCnt):
    result1 = resultCate[i]
    testDf = pd.DataFrame([result1], columns=df_columns)
    cate_df = cate_df.append(testDf, ignore_index=True)

In [24]:
cate_df = cate_df.astype(str)

In [25]:
cate_df

,title,div,cate,cate_link,cate_flag,sub_cate,sub_id
0,SKINCARE,유형별,클렌징,goods/goodsList.do?catCd=4000000000&flag=40000...,4000000001,클렌징 폼,4000000002
1,SKINCARE,유형별,클렌징,goods/goodsList.do?catCd=4000000000&flag=40000...,4000000001,클렌징 오일,4000000003
2,SKINCARE,유형별,클렌징,goods/goodsList.do?catCd=4000000000&flag=40000...,4000000001,클렌징 로션/크림,4000000004
3,SKINCARE,유형별,클렌징,goods/goodsList.do?catCd=4000000000&flag=40000...,4000000001,클렌징 워터,4000000005
4,SKINCARE,유형별,클렌징,goods/goodsList.do?catCd=4000000000&flag=40000...,4000000001,클렌징 티슈,4000000006
5,SKINCARE,유형별,클렌징,goods/goodsList.do?catCd=4000000000&flag=40000...,4000000001,메이크업 리무버,4000000007
6,SKINCARE,유형별,클렌징,goods/goodsList.do?catCd=4000000000&flag=40000...,4000000001,비누,4000000008
7,SKINCARE,유형별,팩/마스크,goods/goodsList.do?catCd=4000000000&flag=40000...,4000000009,워시오프 팩,4000000010
8,SKINCARE,유형별,팩/마스크,goods/goodsList.do?catCd=4000000000&flag=40000...,4000000009,마스크 시트,4000000011
9,SKINCARE,유형별,팩/마스크,goods/goodsList.do?catCd=4000000000&flag=40000...,4000000009,슬리핑 팩,4000000012


In [26]:
cate_df.to_csv("skinfood_SubCategory.csv", index=False)